Tipología y ciclo de vida de los datos: PRÁCTICA I
==================================================

Componentes de la práctica:

 **Jonás Medina Brito** (jmedinabrit@uoc.edu)   
 **Miguel Rafael Esteban Martín** (mestebanmart@uoc.edu)
 
 
## Indice de contenidos

 - [Contexto](#Contexto)
 - [Título](#Título)
 - [Descripción del dataset](#Descripción-del-dataset)
 - [Representación gráfica](#Representación-gráfica)
 - [Contenido](#Contenido)
 - [Agradecimientos](#Agradecimientos)
 - [Inspiración](#Inspiración)
 - [Licencia](#Licencia)
 - [Código](#Código)
 - [Dataset](#Dataset)


## Contexto

### Recogida de datos

Aunque Eurostat tiene una API pública (**1**) que permite obtener los diferentes dataset, en esta práctica hemos decido hacer web scraping  de los datos que se ofrecen en su "_Data browser_" accesible desde un navegador.

> (1)  Eurostat web services: https://ec.europa.eu/eurostat/en/web/main/data/web-services (Ǘltima vista abril del 2020)

Para explicar el estudio que se ha hecho para la obtención de datos se tomará como ejemplo el dataset de "_Componentes del precio de la electricidad para consumidores domésticos_" ([Electricity prices components for household consumers - annual data (from 2007 onwards](https://ec.europa.eu/eurostat/databrowser/view/NRG_PC_204_C__custom_2388428/default/table?lang=en))


![](doc-pract-1-files/dataBrowserInterface.png)

**_Captura de pantalla de la interfaz “Data Browser” del informe   
de los componentes del precio de la electricidad para consumidores  
domésticos_**


El sitio web de Eurostat utiliza HTML dinámico para generación de la vista, esto quiere decir qie el navegador procesa la información y la presenta generando código HTML, y no sólo se limita a representar el código HTML que responde el servidor. Este componente de generación de código por parte del navegador hace complicado el web scraping tradicional.

Utilizando ingeniería inversa y las herramientas de desarrollo del navegador (Firefox), se puede ver cual es la llamada que devuelve los datos que se representan en la tabla dinámica. Un ejemplo es la petición  que es utilizada en la construcción de la tabla con los datos que se muestra en la página web.





## Título

## Descripción del dataset

## Representación gráfica

## Contenido

## Agradecimientos

## Inspiración

## Licencia

## Código

## Dataset